In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import sys
import time

In [ ]:
sys.path.append("/workspace/main-pe/")

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from data.datasets.mpii import MpiiSinglePerson
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

In [ ]:
from model import config
from model import callbacks
from model.utils import pose_format, log

In [ ]:
from experiments.common import exp_init

In [ ]:
# for custom objects
from model.activations import channel_softmax_2d
from model.losses import reconstruction_loss, pose_loss

# datasets

In [ ]:
conf = {
    'pose_blocks': 2,
    'batch_size': 16,
    'dim': 2,
    'n_joints': 16,
    'dataset_name': 'mpii',
    'n_epochs': 2
}

In [ ]:
mpii_path = '/workspace/datasets/mpii'
mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)

In [ ]:
data_tr_mpii = BatchLoader(mpii,
                           ['frame'],
                           ['frame'] + ['pose'] * conf['pose_blocks'],
                           TRAIN_MODE,
                           batch_size=conf['batch_size'],
                           shuffle=False)

In [ ]:
mpii_eval = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)
data_val_mpii = BatchLoader(
    mpii_eval, 
    ['frame'], 
    ['frame', 'pose', 'afmat', 'headsize'], 
    mode=VALID_MODE,
    shuffle=False)

len(data_val_mpii)

## h36m

In [ ]:
conf = {
    'exp_type': 'hybrid_reduced_SAVEFIX_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 16,
    'n_epochs': 60
}

In [ ]:
h36m_path = '/workspace/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [ ]:
# classical i_hat + pose format
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
# validation dataset
h36m_val = BatchLoader(
    h36m, 
    ['frame'], 
    ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
    VALID_MODE, 
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

In [ ]:
log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')
[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

# 1. non nested model

In [ ]:
from model.networks.old_old_mbm import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_NN_OLD_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

## loading

In [ ]:
checkpoint = model_folder + "/model_001.h5"

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
from model.networks.old_old_mbm import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

# 2. Nested

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_NEW_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

## load nested

In [ ]:
# checkpoint = model_folder + "/model_001.h5"
checkpoint = '/workspace/pe_experiments/exp_20190516_1821_2d_NEW_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5'

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

# 3. semi nested

In [ ]:
from model.networks.old_mbm import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_OLD_1b_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

In [ ]:
checkpoint = model_folder + "/model_001.h5"
# checkpoint = '/workspace/pe_experiments/exp_20190516_1028_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5'

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

# 4. Reduced nested

In [ ]:
from model.networks.mbm_reduced import MultiBranchReduced
model = MultiBranchReduced(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_reduced_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

In [ ]:
checkpoint = model_folder + "/model_001.h5"
# checkpoint = '/workspace/pe_experiments/exp_20190517_1419_2d_reduced_SAVE_TEST_NB__2b_bs16/weights_001.h5'

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
checkpoint = '/workspace/pe_experiments/exp_20190517_1419_2d_reduced_SAVE_TEST_NB__2b_bs16/weights_001.h5'

In [ ]:
from model.networks.mbm_reduced import MultiBranchReduced
print("Loading from %s" % checkpoint)
eval_model = MultiBranchReduced(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load_weights(checkpoint)

## eval

In [ ]:
data = data_val_mpii.get_data(1, VALID_MODE)
print(data['frame'].shape)
pred = eval_model.predict(data['frame'])

In [ ]:
import matplotlib.pyplot as plt

n_cols = 2
n_rows = len(data['frame'])
fig = plt.figure(figsize=(8, 75))

print(len(data['frame']))
print(len(pred[0]))

i_img = 1
for i, data_img in enumerate(data['frame']):
    fig.add_subplot(n_rows, n_cols, i_img)
    plt.imshow(data_img)
    
    pred_img = pred[0][i]
    fig.add_subplot(n_rows, n_cols, i_img + 1)
    plt.imshow(pred_img)
    
    i_img += 2
    
plt.show()

## 3d eval

In [ ]:
# data = data_val_h36m.get_data(1, VALID_MODE)
print(x_val.shape)
pred = eval_model.predict(x_val[0])

## debug

In [ ]:
import h5py
import numpy as np
hf = h5py.File(checkpoint, 'r')

In [ ]:
hf.keys()

In [ ]:
a = hf['model_weights']

In [ ]:
print(type(a['pose_model']['stem1_conv']))
b = a['pose_model']['stem1_conv']
list(b.items())

In [ ]:
n1 = a['pose_model']['stem1_conv']
n1 = np.array(n1)
n1.shape

In [ ]:
a['pose_model'].keys()
